In [1]:
from langchain import hub
from typing_extensions import TypedDict
from typing_extensions import Annotated

from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os
from langchain.chat_models import init_chat_model
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [2]:
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

In [6]:
db = SQLDatabase.from_uri("sqlite:///D:/nltexttosql/mlflow.db")

print(db.dialect)
print(db.get_usable_table_names())

# db.run("SELECT * FROM Artist LIMIT 10;")


sqlite
['alembic_version', 'datasets', 'experiment_tags', 'experiments', 'input_tags', 'inputs', 'latest_metrics', 'metrics', 'model_version_tags', 'model_versions', 'params', 'registered_model_aliases', 'registered_model_tags', 'registered_models', 'runs', 'tags', 'trace_info', 'trace_request_metadata', 'trace_tags']


In [5]:
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [6]:
llm = init_chat_model("llama3-8b-8192", model_provider="groq")
query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

C:\Users\saiki\AppData\Local\Temp\ipykernel_15496\4252559610.py:1: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  llm = init_chat_model("llama3-8b-8192", model_provider="groq")
d:\nltexttosql\myvenv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    # print(prompt)
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    print(result)
    return {"query": result["query"]}


In [8]:
write_query({"question": "How many Employees are there?"})

{'query': 'SELECT COUNT(*) FROM Employee'}


{'query': 'SELECT COUNT(*) FROM Employee'}

In [9]:
def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDataBaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

execute_query({"query": "SELECT COUNT(*) FROM Employee;"})

{'result': '[(8,)]'}